In [1]:
!pip install transformers
!pip install tensorflow-text==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 94.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, Bidirectional, TimeDistributed   #layers required for network
from tensorflow.keras.layers import Layer, Conv1D, Softmax, Concatenate ,Dropout, MaxPool1D        #layers required for network
from tensorflow.keras.backend import expand_dims, tile, concatenate, shape, batch_dot, squeeze     #functions required for network
import tensorflow.keras.backend as K                                                               #to build metric
from tensorflow.keras.models import Model                                                          #to build model
from tensorflow.keras.callbacks import TensorBoard                                                 #tensorboard
import tensorflow as tf                                                                            #other functions
from tqdm import tqdm                                                                              #track progress
import numpy as np                                                                                 #for numpy operations
import pickle                                                                            #loading tokenizers
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import BertTokenizer
import math
from numpy import save, load
import tensorflow.nn as nn
import pandas as pd
import time
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [30]:
y_end_test = load("/content/gdrive/MyDrive/Colab Notebooks/BiDAF_SQUAD-master/dataset/y_end_test.npy")
y_start_test = load("/content/gdrive/MyDrive/Colab Notebooks/BiDAF_SQUAD-master/dataset/y_start_test.npy")


test_input_id = load("/content/gdrive/MyDrive/Colab Notebooks/BiDAF_SQUAD-master/dataset/ids_test.npy")
test_input_mask = load("/content/gdrive/MyDrive/Colab Notebooks/BiDAF_SQUAD-master/dataset/mask_test.npy")
test_input_type = load("/content/gdrive/MyDrive/Colab Notebooks/BiDAF_SQUAD-master/dataset/type_test.npy")

In [31]:
len(test_input_id)

15004

In [32]:
rem = 16-(len(test_input_id)%16) if len(test_input_id)%16 != 0 else 0
pad_data = np.zeros((rem,512))

In [33]:
pad_data.shape

(4, 512)

In [34]:
test_input_id = np.concatenate((test_input_id,pad_data),axis=0)
test_input_mask = np.concatenate((test_input_mask,pad_data),axis=0)
test_input_type = np.concatenate((test_input_type,pad_data),axis=0)

In [35]:
# x = np.reshape(test_input_type,(16,int(test_input_type.shape[0]/16),512))
test_input_type.shape

(15008, 512)

In [12]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/modelbase2')
print(model.summary())

Model: "bidaf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Ids (InputLayer)               [(None, 512)]        0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 Type (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 bert (Bert)                    ((None, 52, 768),    109482241   ['Ids[0][0]',                    
                                 (None, 461, 768))                'mask[0][0]',               

In [36]:
fans = []
sttime = time.time()
for i in range(0,len(test_input_type),16):
    start,end = model([test_input_id[i:i+16,:],test_input_mask[i:i+16,:],test_input_type[i:i+16,:]])
    # start = tf.zeros((16,341))
    # end = tf.zeros((16,341))
    # print(start.shape)
    for j in range(16):
        st = tf.math.argmax(start[j])
        en = tf.math.argmax(end[j])
        an = tokenizer.decode(test_input_id[i+j][st:en])
        fans.append(an)
        # print(st,en)
        # print(an)
        # break
    # print(test_input_id[i:i+16,:].shape)

print(f"Execution Time - {time.time()-sttime}")

Execution Time - 1172.7286505699158


In [23]:
data = pd.read_csv('/content/question_answers.csv')

In [46]:
data['Predicted'] =  fans if rem==0 else fans[:-rem]

data.to_csv('/content/final.csv',index=False)

In [25]:
fans

['',
 '',
 'several',
 'entertainment device, the ipod has also been accepted as a business device',
 'dock connector',
 '[CLS] though the ipod was released in 2001, its price and mac - only compatibility caused sales to be relatively slow until 2004. the ipod line came from apple\'s " digital hub " category, when the company',
 'can synchronize entire music libraries or music playlists either automatically or',
 '[CLS] like other digital music players, ipods can serve as external data storage devices. storage capacity varies by model, ranging from 2 gb',
 '2 gb',
 '[CLS] like other digital music players, ipods can serve as external data storage devices. storage capacity varies by model, ranging from 2 gb for the ipod shuffle',
 '128 gb for the ipod touch',
 '2 gb',
 '128 gb',
 'itunes software',
 'of apple macintosh and microsoft',
 'itunes',
 ', separate',
 '',
 'apple store',
 'july 15, 2015',
 '',
 'founder steve wozniak',
 'from 2 gb',
 'game brick',
 'game brick',
 'ipod',
 'jon 

In [26]:
data

,question,theme,paragraph_id,answer,Predicted
0,Which company produces the iPod?,IPod,1,Apple,
1,When was the original iPod released?,IPod,1,"October 23, 2001",
2,How many different types of iPod are currently...,IPod,1,three,several
3,What kind of device is the iPod?,IPod,1,portable media players,"entertainment device, the ipod has also been a..."
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen,dock connector
...,...,...,...,...,...
939,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,ballads,
940,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,hot AC,hot ac
941,What AC format is especially known for playing...,Adult_contemporary_music,642,soft AC,"soft ac stations,"
942,When is the later of the two times that Christ...,Adult_contemporary_music,643,New Year's Day,
